In [1]:
import numpy as np
import pandas as pd
from joblib import dump, load
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KDTree

import os
import sys

## Load Dataset

In [2]:
def reduce_mem_usage(df, verbose=True):
    """ Function iterates through all the columns of a dataframe and modify the data type
        to reduce memory usage.
        Credit to: https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
        Parameters
        ----------
        df : Pandas DataFrame
        verbose: (True) by default, prints out before and after memory usage
        Returns
        -------
        df : Reduced Memory Pandas DataFrame
    """

    if verbose:
        start_mem = df.memory_usage().sum() / 1024**2
        print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    if verbose:
        end_mem = df.memory_usage().sum() / 1024**2
        print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
        print('Decreased by {:.1f}%'.format(
            100 * (start_mem - end_mem) / start_mem))

    return df


def load_dataset():
    """Loads dataset, and combines them into a single dataframe."""
    try:
        fpaths = ['../data/SpotifyAudioFeaturesApril2019.csv',
                  '../data/SpotifyAudioFeaturesNov2018.csv']
        df1 = pd.read_csv(fpaths[0])
        df2 = pd.read_csv(fpaths[1])
        df = pd.concat([df1, df2], ignore_index=True)
        df = reduce_mem_usage(df)  # Reduces Memory Usage

    except Exception as e:
        print(
            'Error Occurred while reading the Spotify dataset: {e}'.format(e))
        raise e

    # Check total sum of rows match
    assert df.shape[0] == (df1.shape[0] + df2.shape[0])

    file_names = [fpath.split('/')[-1] for fpath in fpaths]
    print(
        f'''
        -------------------- SHAPE ---------------------
        DF1 {file_names[0]}: {df1.shape}
        DF2 {file_names[1]}: {df2.shape}
        MERGED DF df: {df.shape}
        ------------------------------------------------
        ''')

    return df


def wrangle(df):

    # Set to avoid SettingWithCopyWarning
    df = df.copy()
    
    # Drop Duplicates
    df = df.drop_duplicates(keep='first')
    
    ## isolating values with a high level of 'speechiness' as they are generally not music but other types of recordings.
    ## converting unwanted values to null then removing those rows
    df['speechiness'] = df['speechiness'].where(df['speechiness'] < .66)
    
    # Drop missing values
    df = df.dropna()

    return df




df = load_dataset()
df.head()

Memory usage of dataframe is 32.04 MB
Memory usage after optimization is: 11.78 MB
Decreased by 63.2%

        -------------------- SHAPE ---------------------
        DF1 SpotifyAudioFeaturesApril2019.csv: (130663, 17)
        DF2 SpotifyAudioFeaturesNov2018.csv: (116372, 17)
        MERGED DF df: (247035, 17)
        ------------------------------------------------
        


,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
0,YG,2RM4jf1Xa9zPgMGRDiht8O,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",0.005821,0.743164,238373,0.339111,0.000000,1,0.081177,-7.679688,1,0.408936,203.8750,4,0.117981,15
1,YG,1tHDG53xJNGsItRA3vfVgs,BAND DRUM (feat. A$AP Rocky),0.024399,0.846191,214800,0.557129,0.000000,8,0.285889,-7.257812,1,0.457031,159.0000,4,0.371094,0
2,R3HAB,6Wosx2euFPMT14UXiWudMy,Radio Silence,0.024994,0.603027,138913,0.723145,0.000000,9,0.082397,-5.890625,0,0.045410,114.9375,4,0.382080,56
3,Chris Cooq,3J2Jpw61sO7l6Hc7qdYV91,Lactose,0.029404,0.799805,125381,0.579102,0.912109,5,0.099426,-12.117188,0,0.070129,123.0000,4,0.641113,0
4,Chris Cooq,2jbYvQCyPgX3CdmAzeVeuS,Same - Original mix,0.000035,0.783203,124016,0.791992,0.877930,7,0.033203,-10.273438,1,0.066101,120.0625,4,0.928223,0


### Create Model using KDTree

In [3]:
# Wrangle Data
wrangled_df = wrangle(df)

print(f'After Wrangling, Shape: {wrangled_df.shape}')
wrangled_df.head()

After Wrangling, Shape: (220804, 17)


,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
0,YG,2RM4jf1Xa9zPgMGRDiht8O,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",0.005821,0.743164,238373,0.339111,0.000000,1,0.081177,-7.679688,1,0.408936,203.8750,4,0.117981,15
1,YG,1tHDG53xJNGsItRA3vfVgs,BAND DRUM (feat. A$AP Rocky),0.024399,0.846191,214800,0.557129,0.000000,8,0.285889,-7.257812,1,0.457031,159.0000,4,0.371094,0
2,R3HAB,6Wosx2euFPMT14UXiWudMy,Radio Silence,0.024994,0.603027,138913,0.723145,0.000000,9,0.082397,-5.890625,0,0.045410,114.9375,4,0.382080,56
3,Chris Cooq,3J2Jpw61sO7l6Hc7qdYV91,Lactose,0.029404,0.799805,125381,0.579102,0.912109,5,0.099426,-12.117188,0,0.070129,123.0000,4,0.641113,0
4,Chris Cooq,2jbYvQCyPgX3CdmAzeVeuS,Same - Original mix,0.000035,0.783203,124016,0.791992,0.877930,7,0.033203,-10.273438,1,0.066101,120.0625,4,0.928223,0


In [4]:
# Select numerical features, relevant to audio track
df_numerics =  wrangled_df.drop(columns=['track_id', 'track_name', 'artist_name'])

# Apply Standard Scaler
# Create a numpy array where the numerical features are scaled to a mean of 0 and a standard deviation of 1.
df_scaled = StandardScaler().fit_transform(df_numerics)

# Apply the model
tree = KDTree(df_scaled)

### Pickle Model 🥒


In [5]:
model_path = '../models/kdt_treemodel.joblib'
dump(tree, model_path)

['../models/kdt_treemodel.joblib']

### Get nearest neighbors distance and indices

In [6]:
# Query the KDTree model for k nearest neighbors.
# (Alternatively) we can use query_radius
# Check sklearn docs 
# https://scikit-learn.org/stable/modules/generated/
# sklearn.neighbors.KDTree.html#sklearn.neighbors.KDTree

num_of_songs = 10
dist, indxs = tree.query(df_scaled, k=num_of_songs+1)

### Check distance of `k=10` similar songs

In [7]:
# Create column names first
column_names = ['Searched_Song']
for i in range(1, num_of_songs+1):
    column_names.append(f'Nearest_Song{i}')
    
# Calculate scores from distances
scores = (1 - ((dist - dist.min()) / (dist.max() - dist.min()))) * 100
dist_df = pd.DataFrame(scores.tolist(), columns=column_names)

# Limit decimal place output to 1
dist_df = dist_df.round(decimals=1)

# Replace Searched Song score with the track_id
dist_df['Searched_Song'] = df.track_id

# Print the shape of the dataframe
print(f'df.shape = {dist_df.shape}')

# Check distances of similar top 10 nearest songs
dist_df.head()

df.shape = (220804, 11)


,Searched_Song,Nearest_Song1,Nearest_Song2,Nearest_Song3,Nearest_Song4,Nearest_Song5,Nearest_Song6,Nearest_Song7,Nearest_Song8,Nearest_Song9,Nearest_Song10
0,2RM4jf1Xa9zPgMGRDiht8O,85.9,85.9,85.7,85.7,85.1,85.0,84.4,83.9,83.7,83.7
1,1tHDG53xJNGsItRA3vfVgs,95.1,88.9,88.7,88.6,88.6,88.6,88.1,88.1,87.9,87.6
2,6Wosx2euFPMT14UXiWudMy,100.0,96.5,96.5,92.6,92.6,92.5,92.5,92.5,92.4,92.3
3,3J2Jpw61sO7l6Hc7qdYV91,95.6,95.6,95.3,95.0,95.0,94.2,93.3,93.0,92.9,92.5
4,2jbYvQCyPgX3CdmAzeVeuS,96.1,95.5,95.3,94.9,94.2,93.8,93.1,93.0,92.9,92.8


### Save Similar Songs

In [8]:
# Making an array of the Track IDs
track_ids = np.array(df.track_id)

# A function that creates list of the each song with its nearest neighbors
def find_similars(track_ids, indxs):
    """For each track in the dataset find similar songs"""
    similars = []
    for row in indxs:
        ids = [track_ids[i] for i in row]
        similars.append(ids)
    return similars

# The first item in each row is a song in the original database.  
# The next 10 rows are its closest neighbors in the KDTree.
nearest_neighbors = find_similars(track_ids, indxs)

# Convert it to dataframe
nearest_neighbors_df = pd.DataFrame(nearest_neighbors, columns=column_names)
nearest_neighbors_df.head()

,Searched_Song,Nearest_Song1,Nearest_Song2,Nearest_Song3,Nearest_Song4,Nearest_Song5,Nearest_Song6,Nearest_Song7,Nearest_Song8,Nearest_Song9,Nearest_Song10
0,2RM4jf1Xa9zPgMGRDiht8O,27j0DacqVJQh0zfeT3ro04,0v2ho4OPJppOu3JXtuiNWG,1wDVA4RdRktGUj1PO9UWry,1JueotTuCnkcTA7tBA5hLR,6gL8GtFF83CljrwbTOWTYa,0xOMrILs6M1Xf9uoihYRy7,68CcjsUbRVus2F1MlcvHTx,0i9ctiWzDtHBuL19lVXv8t,2OIZgCdPVPHAV8ZJAkEmAI,5W0TmFbf99HsjItflpvu2y
1,1tHDG53xJNGsItRA3vfVgs,3pU68iUbatcIjQD7PlIzFX,0NuZTXfxEt9cYcpUcnf6Jw,1MyEbHLZNqRgRP0XvAUrjB,0tZvlW8YxwnPS7Ui7pzF9q,3zpYoc5mkMwaY4JjNiitf9,0w76exF9cYIDXc1tkZGPFw,7vj9Rur6URu4sY5JoLGHP0,7k72R754CDdtZnWd4TORo4,6tFQ816K1IcSkLdpI2OTTh,1XJxfmKVIddQhlaTkAnJui
2,6Wosx2euFPMT14UXiWudMy,6z3OqZBR4v3w6s0w6dSVhF,536LtAJlONCorMQe539cKy,6UeZN3zstB5uu8JJM2eUM4,5Aiaju6nfXu6lTJNwfcndZ,7BrHYVzz9NA07DXo76sjdR,3KHPq5Im4kC9JPXWz6CwbT,5qjMQRC7fr5Z7pkytUh16b,4Mb2vNoUJ28dOSNwK1lhXx,1rvTKiLbFJuVvdaxnjlDtz,58iZeZLWATl8A9oUQnjlbj
3,3J2Jpw61sO7l6Hc7qdYV91,1jFL774GMnBl5Esa9SN4rL,5IsIBQ58FTGKxCHOkWOqio,0mAfMyv2YEm6eek8LX7hdV,01qc7HHvJdcNviyizhuCSI,7GhxPuxjwAbKEiQFMP5Mxd,2F4kolGzp9QN30sR0U5ord,1dcG1iBWoZzTj5zBWgkFFa,68QJVJJRoFXIQT6VW0dOTI,4lYNaHwkcKugUbai11GK3J,0prYWdIWaPR4xOCbdy00dQ
4,2jbYvQCyPgX3CdmAzeVeuS,4XAxQKMOR9scxNT43lBGgL,5K3nukdcaKk3dnDiVoDaWU,58StYU2nRzsyPPq9rXXJmK,5A5QbhltFlUwdgi0CFdHsZ,0HrMEvZHO45VboRZcpId8z,2K9W9VYOCUnHVAEfPoNzO8,0sRFDf3y2ELoP7YvnQDVnI,1MiVscnzTTwr84qHIZ6HXO,3uzWxX43P7RlUx9VjOugPS,09ZiRPA8PBNvCgfq7RhY06


In [9]:
nearest_neighbors_df.shape

(220804, 11)

## Drop duplicate rows

In [10]:
nearest_neighbors_df[nearest_neighbors_df.duplicated()].sample(1, random_state=42)

,Searched_Song,Nearest_Song1,Nearest_Song2,Nearest_Song3,Nearest_Song4,Nearest_Song5,Nearest_Song6,Nearest_Song7,Nearest_Song8,Nearest_Song9,Nearest_Song10
124387,1oRRLczVT5KkZkPHXG70n7,4VKOPmEcSbejMpIMwDXJd6,1Oa0du1SpDw3iliDYE7mcD,63Jn8YOTbmHR70PTdvCe8p,3sTB8tWNFTLl6tGLwsqSWc,1y48A57Q8EN1cjMrJEsN1N,39r3UkUOYMh9pPQmKSZnuE,6mjjypSqiieMoYr0t7hSqb,7BNs7TN3FewRiqp91CluNC,2Epo0BPFWIYHbVDnXVwcDH,6cvRd7Q1KIMflD2j8xqIgC


In [11]:
nearest_neighbors_df.query('Searched_Song == "1oRRLczVT5KkZkPHXG70n7"')

,Searched_Song,Nearest_Song1,Nearest_Song2,Nearest_Song3,Nearest_Song4,Nearest_Song5,Nearest_Song6,Nearest_Song7,Nearest_Song8,Nearest_Song9,Nearest_Song10
109797,1oRRLczVT5KkZkPHXG70n7,4VKOPmEcSbejMpIMwDXJd6,1Oa0du1SpDw3iliDYE7mcD,63Jn8YOTbmHR70PTdvCe8p,3sTB8tWNFTLl6tGLwsqSWc,1y48A57Q8EN1cjMrJEsN1N,39r3UkUOYMh9pPQmKSZnuE,6mjjypSqiieMoYr0t7hSqb,7BNs7TN3FewRiqp91CluNC,2Epo0BPFWIYHbVDnXVwcDH,6cvRd7Q1KIMflD2j8xqIgC
124387,1oRRLczVT5KkZkPHXG70n7,4VKOPmEcSbejMpIMwDXJd6,1Oa0du1SpDw3iliDYE7mcD,63Jn8YOTbmHR70PTdvCe8p,3sTB8tWNFTLl6tGLwsqSWc,1y48A57Q8EN1cjMrJEsN1N,39r3UkUOYMh9pPQmKSZnuE,6mjjypSqiieMoYr0t7hSqb,7BNs7TN3FewRiqp91CluNC,2Epo0BPFWIYHbVDnXVwcDH,6cvRd7Q1KIMflD2j8xqIgC


In [12]:
final_df1 = nearest_neighbors_df[~nearest_neighbors_df.duplicated()]
final_df1.shape

(219856, 11)

In [14]:
final_df1.head()

,Searched_Song,Nearest_Song1,Nearest_Song2,Nearest_Song3,Nearest_Song4,Nearest_Song5,Nearest_Song6,Nearest_Song7,Nearest_Song8,Nearest_Song9,Nearest_Song10
0,2RM4jf1Xa9zPgMGRDiht8O,27j0DacqVJQh0zfeT3ro04,0v2ho4OPJppOu3JXtuiNWG,1wDVA4RdRktGUj1PO9UWry,1JueotTuCnkcTA7tBA5hLR,6gL8GtFF83CljrwbTOWTYa,0xOMrILs6M1Xf9uoihYRy7,68CcjsUbRVus2F1MlcvHTx,0i9ctiWzDtHBuL19lVXv8t,2OIZgCdPVPHAV8ZJAkEmAI,5W0TmFbf99HsjItflpvu2y
1,1tHDG53xJNGsItRA3vfVgs,3pU68iUbatcIjQD7PlIzFX,0NuZTXfxEt9cYcpUcnf6Jw,1MyEbHLZNqRgRP0XvAUrjB,0tZvlW8YxwnPS7Ui7pzF9q,3zpYoc5mkMwaY4JjNiitf9,0w76exF9cYIDXc1tkZGPFw,7vj9Rur6URu4sY5JoLGHP0,7k72R754CDdtZnWd4TORo4,6tFQ816K1IcSkLdpI2OTTh,1XJxfmKVIddQhlaTkAnJui
2,6Wosx2euFPMT14UXiWudMy,6z3OqZBR4v3w6s0w6dSVhF,536LtAJlONCorMQe539cKy,6UeZN3zstB5uu8JJM2eUM4,5Aiaju6nfXu6lTJNwfcndZ,7BrHYVzz9NA07DXo76sjdR,3KHPq5Im4kC9JPXWz6CwbT,5qjMQRC7fr5Z7pkytUh16b,4Mb2vNoUJ28dOSNwK1lhXx,1rvTKiLbFJuVvdaxnjlDtz,58iZeZLWATl8A9oUQnjlbj
3,3J2Jpw61sO7l6Hc7qdYV91,1jFL774GMnBl5Esa9SN4rL,5IsIBQ58FTGKxCHOkWOqio,0mAfMyv2YEm6eek8LX7hdV,01qc7HHvJdcNviyizhuCSI,7GhxPuxjwAbKEiQFMP5Mxd,2F4kolGzp9QN30sR0U5ord,1dcG1iBWoZzTj5zBWgkFFa,68QJVJJRoFXIQT6VW0dOTI,4lYNaHwkcKugUbai11GK3J,0prYWdIWaPR4xOCbdy00dQ
4,2jbYvQCyPgX3CdmAzeVeuS,4XAxQKMOR9scxNT43lBGgL,5K3nukdcaKk3dnDiVoDaWU,58StYU2nRzsyPPq9rXXJmK,5A5QbhltFlUwdgi0CFdHsZ,0HrMEvZHO45VboRZcpId8z,2K9W9VYOCUnHVAEfPoNzO8,0sRFDf3y2ELoP7YvnQDVnI,1MiVscnzTTwr84qHIZ6HXO,3uzWxX43P7RlUx9VjOugPS,09ZiRPA8PBNvCgfq7RhY06


In [15]:
final_df1[final_df1['Searched_Song'].duplicated()].sample(1, random_state=42)

,Searched_Song,Nearest_Song1,Nearest_Song2,Nearest_Song3,Nearest_Song4,Nearest_Song5,Nearest_Song6,Nearest_Song7,Nearest_Song8,Nearest_Song9,Nearest_Song10
186809,4W6cGrf1UKI8Z4a60thGKU,2eFq2nccbEHmK9b1TEVgT6,2fRC9nVbZCfEzkow3Dshdj,1m9H6jcDNNVIistQ9C9kuP,1cuu1CWqLGbEowgJR15cjl,50vmPRR1FS0qzTz0WGadqn,3IMeGeV9ls6L43MWVIsjFs,4eKn4v8JcaSvWRDU1uoWWw,1lmk3S276n3TMFfwnG5yqu,7dX4zly1R8h8GMJpISsKkA,6MhzEgKtdj1jDaeophCJtD


In [16]:
final_df1.query('Searched_Song == "4W6cGrf1UKI8Z4a60thGKU"')

,Searched_Song,Nearest_Song1,Nearest_Song2,Nearest_Song3,Nearest_Song4,Nearest_Song5,Nearest_Song6,Nearest_Song7,Nearest_Song8,Nearest_Song9,Nearest_Song10
55833,4W6cGrf1UKI8Z4a60thGKU,0eFh17Y6XuD3SNyceRODzh,4IPKfv5dbRdfQujHMtsPcc,0zOULJjaszyWWMuds8O7kd,1NsbdxXsMS3m8Q79P6GWLo,6cDAqCOhGPVPilQRl3wPem,20YOkoyY7RT3gNpdr066CL,639d3rulv2OZzSFArAUhNu,4NtguR2CXEL6Mv6N7awpl8,4LRNfo83sqH58BsFWMO9jK,6MgctgFKCDlaseBuK2UBXr
186809,4W6cGrf1UKI8Z4a60thGKU,2eFq2nccbEHmK9b1TEVgT6,2fRC9nVbZCfEzkow3Dshdj,1m9H6jcDNNVIistQ9C9kuP,1cuu1CWqLGbEowgJR15cjl,50vmPRR1FS0qzTz0WGadqn,3IMeGeV9ls6L43MWVIsjFs,4eKn4v8JcaSvWRDU1uoWWw,1lmk3S276n3TMFfwnG5yqu,7dX4zly1R8h8GMJpISsKkA,6MhzEgKtdj1jDaeophCJtD


In [17]:
final_df2 = final_df1.drop_duplicates(subset=['Searched_Song'], keep='first')
final_df2.shape

(130150, 11)

### Save it as csv file

In [18]:
def save_data_frame_as_csv(df=None, filename=None):
    """
    Saves data frame to csv format
    
    Parameters
    ----------
    df: Pandas DataFrame
    filename: File path or name
    
    Returns
    -------
    csv file
    """
    try:
        if not filename.endswith('.csv'):
            filename += '.csv'
        df.to_csv(filename, index=False)
        print(f"Data Frame saved @:{filename}")
    except Exception as e:
        print("Data Frame couldn't be saved: ", sys.exc_info()[0])
        raise
        
save_data_frame_as_csv(final_df2, '../data/spotify_dataset_recommendations.csv')

Data Frame saved @:../data/spotify_dataset_recommendations.csv


### Save it as JSON file

In [19]:
def save_data_frame_as_json(df=None, filename=None, orient="records"):
    """
    Saves data frame to JSON format
    
    Parameters
    ----------
    df: Pandas DataFrame
    filename: File path or name
    
    Returns
    -------
    JSON file
    """
    try:
        if not filename.endswith('.json'):
            filename += '.json'
        df.to_json(filename, orient=orient)
        print(f"Data Frame saved @:{filename}")
    except Exception as e:
        print("Data Frame couldn't be saved: ", sys.exc_info()[0])
        raise
save_data_frame_as_json(final_df2, '../data/spotify_dataset_recommendations.json')

Data Frame saved @:../data/spotify_dataset_recommendations.json


In [20]:
json_df = pd.read_json('../data/spotify_dataset_recommendations.json')
json_df.head()

,Searched_Song,Nearest_Song1,Nearest_Song2,Nearest_Song3,Nearest_Song4,Nearest_Song5,Nearest_Song6,Nearest_Song7,Nearest_Song8,Nearest_Song9,Nearest_Song10
0,2RM4jf1Xa9zPgMGRDiht8O,27j0DacqVJQh0zfeT3ro04,0v2ho4OPJppOu3JXtuiNWG,1wDVA4RdRktGUj1PO9UWry,1JueotTuCnkcTA7tBA5hLR,6gL8GtFF83CljrwbTOWTYa,0xOMrILs6M1Xf9uoihYRy7,68CcjsUbRVus2F1MlcvHTx,0i9ctiWzDtHBuL19lVXv8t,2OIZgCdPVPHAV8ZJAkEmAI,5W0TmFbf99HsjItflpvu2y
1,1tHDG53xJNGsItRA3vfVgs,3pU68iUbatcIjQD7PlIzFX,0NuZTXfxEt9cYcpUcnf6Jw,1MyEbHLZNqRgRP0XvAUrjB,0tZvlW8YxwnPS7Ui7pzF9q,3zpYoc5mkMwaY4JjNiitf9,0w76exF9cYIDXc1tkZGPFw,7vj9Rur6URu4sY5JoLGHP0,7k72R754CDdtZnWd4TORo4,6tFQ816K1IcSkLdpI2OTTh,1XJxfmKVIddQhlaTkAnJui
2,6Wosx2euFPMT14UXiWudMy,6z3OqZBR4v3w6s0w6dSVhF,536LtAJlONCorMQe539cKy,6UeZN3zstB5uu8JJM2eUM4,5Aiaju6nfXu6lTJNwfcndZ,7BrHYVzz9NA07DXo76sjdR,3KHPq5Im4kC9JPXWz6CwbT,5qjMQRC7fr5Z7pkytUh16b,4Mb2vNoUJ28dOSNwK1lhXx,1rvTKiLbFJuVvdaxnjlDtz,58iZeZLWATl8A9oUQnjlbj
3,3J2Jpw61sO7l6Hc7qdYV91,1jFL774GMnBl5Esa9SN4rL,5IsIBQ58FTGKxCHOkWOqio,0mAfMyv2YEm6eek8LX7hdV,01qc7HHvJdcNviyizhuCSI,7GhxPuxjwAbKEiQFMP5Mxd,2F4kolGzp9QN30sR0U5ord,1dcG1iBWoZzTj5zBWgkFFa,68QJVJJRoFXIQT6VW0dOTI,4lYNaHwkcKugUbai11GK3J,0prYWdIWaPR4xOCbdy00dQ
4,2jbYvQCyPgX3CdmAzeVeuS,4XAxQKMOR9scxNT43lBGgL,5K3nukdcaKk3dnDiVoDaWU,58StYU2nRzsyPPq9rXXJmK,5A5QbhltFlUwdgi0CFdHsZ,0HrMEvZHO45VboRZcpId8z,2K9W9VYOCUnHVAEfPoNzO8,0sRFDf3y2ELoP7YvnQDVnI,1MiVscnzTTwr84qHIZ6HXO,3uzWxX43P7RlUx9VjOugPS,09ZiRPA8PBNvCgfq7RhY06


### Save it as SQL database

In [25]:
def save_data_as_sqlite_database(df, sql_uri, table_name, index=True, index_label=None, verbose=True):
    """
    Saves data frame to SQLite database
    
    Parameters
    ----------
    df: Pandas DataFrame
    filename: File path or name
    
    Returns
    -------
    db.sqlite3
    """
    try:
        if not sql_uri.endswith('.sqlite3'):
            sql_uri += '.sqlite3'
        if not sql_uri.startswith('sqlite:///'):
            raise ValueError('Database URL should start with "sqlite:///"')
        
        from sqlalchemy import create_engine
        engine = create_engine(sql_uri)
        print(f"Data Frame saved @:{sql_uri[10:]}")
        df.to_sql(table_name, con=engine, index=index, index_label=index_label, if_exists='replace')
        
        if verbose:
            sql = f"SELECT * FROM {table_name} LIMIT 10;"
            print(pd.read_sql(sql, con=engine))
    except Exception as e:
        print("Data Frame couldn't be saved: ", sys.exc_info()[0])
        raise

sql_uri = "sqlite:///../data/db.sqlite3"
table_name = "recommendations"
save_data_as_sqlite_database(final_df2, sql_uri, table_name, index=True, index_label=None, verbose=True)

Data Frame saved @:../data/db.sqlite3
   index           Searched_Song           Nearest_Song1  \
0      0  2RM4jf1Xa9zPgMGRDiht8O  27j0DacqVJQh0zfeT3ro04   
1      1  1tHDG53xJNGsItRA3vfVgs  3pU68iUbatcIjQD7PlIzFX   
2      2  6Wosx2euFPMT14UXiWudMy  6z3OqZBR4v3w6s0w6dSVhF   
3      3  3J2Jpw61sO7l6Hc7qdYV91  1jFL774GMnBl5Esa9SN4rL   
4      4  2jbYvQCyPgX3CdmAzeVeuS  4XAxQKMOR9scxNT43lBGgL   
5      5  26Y1lX7ZOpw9Ql3gGAlqLK  4naIUyujaQ5dtxkWQnwJA4   
6      6  5eIyK73BrxHLnly4F9PWqg  4y6RAqWKiLFpIstFNkx4ao   
7      7  13Mf2ZBpfNkgWJowvM5hXh  6Zn3oEkBJEgoXecFDuwpe2   
8      8  7BQaRTHk44DkMhIVNcXy2D  7iHpX7m9owsqawBy008yvF   
9      9  049RxG2laEl9U1PGYeIqLV  3ZNV1H2YO7JwayxVjA5mKm   

            Nearest_Song2           Nearest_Song3           Nearest_Song4  \
0  0v2ho4OPJppOu3JXtuiNWG  1wDVA4RdRktGUj1PO9UWry  1JueotTuCnkcTA7tBA5hLR   
1  0NuZTXfxEt9cYcpUcnf6Jw  1MyEbHLZNqRgRP0XvAUrjB  0tZvlW8YxwnPS7Ui7pzF9q   
2  536LtAJlONCorMQe539cKy  6UeZN3zstB5uu8JJM2eUM4  5Aiaju6nfXu6lTJNwfc